In [23]:
from bs4 import BeautifulSoup
import requests

In [24]:
url = 'https://attack.mitre.org/software/'
page = requests.get(url)

In [25]:
if page.status_code == 200:
    soup = BeautifulSoup(page.text,'html')
else:
    print("request failed")

In [26]:
table_data = soup.find("table")

In [27]:
table_body = table_data.find("tbody")

In [28]:
result = []
for table_row in table_body.findAll("tr"):
    table_cols = table_row.findAll("td")
    result.append({"id":table_cols[0].text.replace("\n", "").strip(), "name":table_cols[1].text.replace("\n", "").strip(),"Associated Software":table_cols[2].text.replace("\n", "").strip(),"Description":table_cols[3].text.replace("\n", "").strip()})


In [29]:
result

[{'id': 'S0066',
  'name': '3PARA RAT',
  'Associated Software': '',
  'Description': '3PARA RAT is a remote access tool (RAT) programmed in C++ that has been used by Putter Panda.'},
 {'id': 'S0065',
  'name': '4H RAT',
  'Associated Software': '',
  'Description': '4H RAT is malware that has been used by Putter Panda since at least 2007.'},
 {'id': 'S0677',
  'name': 'AADInternals',
  'Associated Software': '',
  'Description': 'AADInternals is a PowerShell-based framework for administering, enumerating, and exploiting Azure Active Directory. The tool is publicly available on GitHub.'},
 {'id': 'S0469',
  'name': 'ABK',
  'Associated Software': '',
  'Description': 'ABK is a downloader that has been used by BRONZE BUTLER since at least 2019.'},
 {'id': 'S1061',
  'name': 'AbstractEmu',
  'Associated Software': '',
  'Description': 'AbstractEmu is mobile malware that was first seen in Google Play and other third-party stores in October 2021. It was discovered in 19 Android application

In [30]:
all_links = []

for table_row in table_body.findAll("tr"):
    table_col = table_row.findAll("td")
    links_in_col = table_col[1].findAll("a")
    
    # Extract href attribute
    for link in links_in_col:
        href = link.get("href")
        if href:
            all_links.append(href)


print(all_links)
# print(all_links[0])


['/software/S0066', '/software/S0065', '/software/S0677', '/software/S0469', '/software/S1061', '/software/S1000', '/software/S1125', '/software/S1028', '/software/S0202', '/software/S0552', '/software/S0309', '/software/S0045', '/software/S0440', '/software/S0331', '/software/S0092', '/software/S1095', '/software/S1129', '/software/S0319', '/software/S1025', '/software/S0504', '/software/S0525', '/software/S0304', '/software/S0524', '/software/S0310', '/software/S1074', '/software/S0292', '/software/S0422', '/software/S0584', '/software/S0622', '/software/S0456', '/software/S0099', '/software/S0540', '/software/S0073', '/software/S0373', '/software/S1087', '/software/S0110', '/software/S0438', '/software/S0347', '/software/S1029', '/software/S0129', '/software/S0640', '/software/S0473', '/software/S1053', '/software/S0344', '/software/S0638', '/software/S0414', '/software/S0475', '/software/S0093', '/software/S0031', '/software/S0606', '/software/S0245', '/software/S0642', '/software/

In [36]:
def fetching_meta_information(soup2):
    result2 = []

    try:
        # card_body = soup2.find("div", class_="card-body").text
        card_body = soup2.find("div", class_="card-body")
        if card_body:
            divs = soup2.findAll('div', class_="row card-data")

            data = []
            for div in divs:
                data.append(div.get_text(strip=True))
            
            # Ensure that data contains all required elements
            if len(data) >= 6:
                result2.append({
                    "ID": data[0] if data[0] else "N/A",
                    "Type": data[1] if data[1] else "N/A",
                    "Platforms": data[2] if data[2] else "N/A",
                    "Version": data[3] if data[3] else "N/A",
                    "Created": data[4] if data[4] else "N/A",
                    "Last Modified": data[5] if data[5] else "N/A"
                })
            else:
                result2.append({
                    "ID": data[0] if len(data) > 0 else "N/A",
                    "Type": data[1] if len(data) > 1 else "N/A",
                    "Platforms": data[2] if len(data) > 2 else "N/A",
                    "Version": data[3] if len(data) > 3 else "N/A",
                    "Created": data[4] if len(data) > 4 else "N/A",
                    "Last Modified": data[5] if len(data) > 5 else "N/A"
                })
        else:
            print("No card-body found")
    except Exception as e:
        print(f"An error occurred: {e}")

    return(result2)


In [45]:
import sqlite3

def insert_data_into_table(data):
    # Connect to SQLite database (or create it if it doesn't exist)
    conn = sqlite3.connect('RHYM(database).db')
    cursor = conn.cursor()

    # Create the table if it doesn't exist
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS mitre_malware_testing3 (
            id TEXT,
            name TEXT,
            Associated_Software TEXT,
            Description TEXT
        )
    ''')

    # Insert data into the table
    for item in data:
        id = item['id']
        name = item['name']
        associated_software = item['Associated Software']
        description = item['Description']

        cursor.execute('''
            INSERT INTO mitre_malware_testing3 (id, name, Associated_Software, Description)
            VALUES (?, ?, ?, ?)
        ''', (id, name, associated_software, description))

    # Commit the transaction and close the connection
    conn.commit()
    conn.close()

In [40]:
# for link in all_links:
#     link = "https://attack.mitre.org"+link
#     url2 = link
#     page2 = requests.get(url2)
    

#     if page2.status_code == 200:
#         soup2 = BeautifulSoup(page2.text,'html')
#         data = fetching_meta_information(soup2)
#         # insert_data_into_table(data)
#     else:
#         print("request not found")

    

    
    
    

In [46]:
insert_data_into_table(result)

In [49]:
def fetch_all_data():
    # Connect to SQLite database
    conn = sqlite3.connect('RHYM(database).db')
    cursor = conn.cursor()

    # Fetch all records from the table
    cursor.execute('SELECT * FROM mitre_malware_testing3')
    rows = cursor.fetchall()

    # Print the results
    for row in rows:
        print(row)

    # Close the connection
    conn.close()

In [50]:
fetch_all_data()

('S0066', '3PARA RAT', '', '3PARA RAT is a remote access tool (RAT) programmed in C++ that has been used by Putter Panda.')
('S0065', '4H RAT', '', '4H RAT is malware that has been used by Putter Panda since at least 2007.')
('S0677', 'AADInternals', '', 'AADInternals is a PowerShell-based framework for administering, enumerating, and exploiting Azure Active Directory. The tool is publicly available on GitHub.')
('S0469', 'ABK', '', 'ABK is a downloader that has been used by BRONZE BUTLER since at least 2019.')
('S1061', 'AbstractEmu', '', 'AbstractEmu is mobile malware that was first seen in Google Play and other third-party stores in October 2021. It was discovered in 19 Android applications, of which at least 7 abused known Android exploits for obtaining root permissions. AbstractEmu was observed primarily impacting users in the United States, however victims are believed to be across a total of 17 countries.')
('S1000', 'ACAD/Medre.A', '', 'ACAD/Medre.A is a worm that steals operat